In [63]:
from bs4 import BeautifulSoup
import datetime
import requests
import pandas as pd
import numpy as np

header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
town_to_nickname_dict = {'Atlanta': 'Hawks',
                         'Boston': 'Celtics',
                         'Brooklyn': 'Nets',
                         'Charlotte': 'Hornets',
                         'Chicago': 'Bulls',
                         'Cleveland': 'Cavaliers',
                         'Dallas': 'Mavericks',
                         'Denver': 'Nuggets',
                         'Detroit': 'Pistons',
                         'Golden State': 'Warriors',
                         'Houston': 'Rockets',
                         'Indiana': 'Pacers',
                         'L.A. Clippers': 'Clippers',
                         'L.A. Lakers': 'Lakers',
                         'Memphis': 'Grizzlies',
                         'Miami': 'Heat',
                         'Milwaukee': 'Bucks',
                         'Minnesota': 'Timberwolves',
                         'New Orleans': 'Pelicans',
                         'New York': 'Knicks',
                         'Oklahoma City': 'Thunder',
                         'Orlando': 'Magic',
                         'Philadelphia': '76ers',
                         'Phoenix': 'Suns',
                         'Portland': 'Trail Blazers',
                         'Sacramento': 'Kings',
                         'San Antonio': 'Spurs',
                         'Toronto': 'Raptors',
                         'Utah': 'Jazz',
                         'Washington': 'Wizards'
                        }
nickname_to_acronym_dict = {'Hawks': 'Atl',
                         'Celtics': 'Bos',
                         'Nets': 'BKN',
                         'Hornets': 'Cha',
                         'Bulls': 'Chi',
                         'Cavaliers': 'CLE',
                         'Mavericks': 'DAL',
                         'Nuggets': 'DEN',
                         'Pistons': 'DET',
                         'Warriors': 'GS',
                         'Rockets': 'HOU',
                         'Pacers': 'IND',
                         'Clippers': 'LAC',
                         'Lakers': 'LAL',
                         'Grizzlies': 'MEM',
                         'Heat': 'MIA',
                         'Bucks': 'MIL',
                         'Timberwolves': 'MIN',
                         'Pelicans': 'NO',
                         'Knicks': 'NY',
                         'Thunder': 'OKC',
                         'Magic': 'ORL',
                         '76ers': 'PHI',
                         'Suns': 'PHX',
                         'Trail Blazers': 'POR',
                         'Kings': 'SAC',
                         'Spurs': 'SA',
                         'Raptors': 'TOR',
                         'Jazz': 'UTA',
                         'Wizards': 'WSH'
                        }

In [72]:
def initial_538_predictions(game_date):
    """Creates dataframe with initial values from 538
    Args:
        week_num: week number of games
    Returns:
        dataframe of 538 predictions
    """
    df_columns = ['date', 'team', '538 win%']
    df = pd.DataFrame(columns=df_columns)
    
    output_file = 'z:\python_projects\aaa.exe'
    year = 2019
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2020-nba-predictions/games/'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    # Finding year of predictions
    for timestamp in soup.findAll('div', attrs={'class': 'container'}):
        for year in timestamp.findAll('div', attrs={'id': 'intro'}):
            year = int(year.h1.get_text()[0:4])

    # Gathering data
    for day_group in soup.findAll('div', attrs={'class': 'day-group'}):
        for day_of_year in day_group.findAll('section', attrs={'class': 'day'}):
            date = day_of_year.h3.get_text().strip()
            date = date.split(', ')[1]
            date = date[0:3] + ' ' + date[-2:].strip() + ' ' + str(year)
            date = datetime.datetime.strptime(date, '%b %d %Y')
            year = date.year
            month = date.month
            day = date.day
            date = date.strftime('%m/%d/%Y')
            
            if date == game_date:
                # Finding data for each game
                for game in day_of_year.findAll('tbody', attrs={'class': 'ie10up'}):
                    for num_teams, data in enumerate(game.findAll('tr', attrs={'class': 'tr team'})): 
                        team_acronym = data['data-team']
                        squad = data.find('td', attrs={'class': 'td text team '+team_acronym})
                        win_percentage = data.find('td', attrs={'class': 'td number chance'}).get_text().strip()
                        if squad:
                            team = squad.get_text().strip()
                            result = ''
                    
                            if num_teams == 0:
                                df = df.append(pd.Series([date, team, win_percentage], index=df.columns),
                                                          ignore_index=True)
                            else:
                                df = df.append(pd.Series(['', team, win_percentage], index=df.columns),
                                                          ignore_index=True)
                        
                        

    df.drop_duplicates(inplace=True)
    
    return df, year
    
def getting_spreadsheet(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))
def separating_games(df):
    """Separates old games and new games
    Args:
        df: dataframe of data
    Returns:
        old_games: games with results already
        current_game: games without results
    """
    old_games = df[df['result'] != '']
    old_games_index = df[df['result'] != ''].index 
    current_games = df.drop(old_games_index)
    return old_games, current_games
def predictions_538(game_date):
    """Loads 538 predictions
    Args:
        week_num: week number of games
    Returns:
        df: dataframe with 538 predictions
        year: year of games. Needed for Vegas odds
    """
    year = 2019
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2020-nba-predictions/games/'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    cols = ['date', 'team', '538 win%']
    blank_rows = ['']*(3)
    cols.extend(blank_rows)
    predictions = pd.DataFrame(columns=cols)

    # Finding year of predictions
    for timestamp in soup.findAll('div', attrs={'class': 'container'}):
        for year in timestamp.findAll('div', attrs={'id': 'intro'}):
            year = int(year.h1.get_text()[0:4])

    # Gathering data
    for day_group in soup.findAll('div', attrs={'class': 'day-group'}):
        for day_of_year in day_group.findAll('section', attrs={'class': 'day'}):
            date = day_of_year.h3.get_text().strip()
            date = date.split(', ')[1]
            date = date[0:3] + ' ' + date[-2:].strip() + ' ' + str(year)
            date = datetime.datetime.strptime(date, '%b %d %Y')
            year = date.year
            month = date.month
            day = date.day
            date = date.strftime('%m/%d/%Y')

            if date == game_date:
                # Finding data for each game
                for game in day_of_year.findAll('tbody', attrs={'class': 'ie10up'}):
                    for num_teams, data in enumerate(game.findAll('tr', attrs={'class': 'tr team'})): 
                        team_acronym = data['data-team']
                        squad = data.find('td', attrs={'class': 'td text team '+team_acronym})
                        win_percentage = data.find('td', attrs={'class': 'td number chance'}).get_text().strip()
                        if squad:
                            team = squad.get_text().strip()
                            result = ''
                            
                            if num_teams == 0:
                                row_data = [date, team, win_percentage]
                                row_data.extend(blank_rows)
                                predictions = predictions.append(pd.Series(row_data, index=predictions.columns), 
                                                                 ignore_index=True)
                            else:
                                row_data = ['', team, win_percentage]
                                row_data.extend(blank_rows)                                
                                predictions = predictions.append(pd.Series(row_data, index=predictions.columns), 
                                                                 ignore_index=True)
                        
                            
    return predictions, year
def predictions_fox(df):
    """Loads fox predictions
    Args:
    df: dataframe of data that will be merged with
    Returns:
        df: dataframe including Fox predictions
    """
    predictions_columns = ['week', 'team', 'Fox win%']
    predictions = pd.DataFrame(columns=predictions_columns)

    year = 2019
    result = ''
    URL = 'https://www.foxsports.com/nfl/predictions'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    for week in soup.findAll('span', attrs={'class': 'wisbb_pageInfoSecondaryText'}):
        week_num = int(week.get_text().strip()[5:])


    for matchup in soup.findAll('div', attrs={'class': 'wisbb_predictionChip'}):
        for num, team_name in enumerate(matchup.findAll('span', attrs={'class': 'wisbb_teamName'})):
            if num == 0:
                away_team = team_name.get_text().strip()
            elif num == 1:
                home_team = team_name.get_text().strip()
            else:
                continue

        for num, fox_predictions in enumerate(matchup.findAll('span', attrs={'class': 'wisbb_predData'})):
            prediction_text = fox_predictions.get_text()
            team_acronym = fox_predictions.get_text()[0:3].strip()
            if team_acronym in acronym_to_town_dict and len(prediction_text)>4 and num ==1:
                if acronym_to_town_dict[team_acronym] == nickname_to_town_dict[away_team]:
                    away_win_percentage_int = int(round(float(prediction_text.split('(')[1][:-2])))
                    away_win_percentage_string = str(away_win_percentage_int) + '%'
                    home_win_percentage_int = 100 - away_win_percentage_int
                    home_win_percentage_string = str(home_win_percentage_int) + '%'

                elif acronym_to_town_dict[team_acronym] == nickname_to_town_dict[home_team]:
                    home_win_percentage_int = int(round(float(prediction_text.split('(')[1][:-2])))
                    home_win_percentage_string = str(home_win_percentage_int) + '%'
                    away_win_percentage_int = 100 - home_win_percentage_int
                    away_win_percentage_string = str(away_win_percentage_int) + '%'
                else:
                    continue

                predictions = predictions.append(pd.Series([week_num, nickname_to_town_dict[away_team], away_win_percentage_string],
                                             index=predictions.columns), ignore_index=True)
                predictions = predictions.append(pd.Series([week_num, nickname_to_town_dict[home_team], home_win_percentage_string],
                                             index=predictions.columns), ignore_index=True)

    df = pd.merge(predictions, df)
    
    return df
def loading_odds(df, year):
    """Loads odds to spreadsheet
    Args:
        df: dataframe that odds will be written to
        year: year that will be used to gather the dates of the games
    Returns:
        dataframe of datawith odds
    """
    names = ['Open', 'odds','Westgate','MGM Mirage', 'betMGM',
             'William Hill', 'CG Technology', 'Circa Sports','Stations']
    book = pd.DataFrame(columns=names)
    output_file = 'z:\python_projects\aaa.exe'
    temp_away_list = []
    temp_home_list = []
    teams_list = []
    dates_list = []
    URL = 'http://www.vegasinsider.com/nba/odds/las-vegas/money/'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    for gameboard in soup.findAll('table', attrs={'class': 'viBodyContainerTble'}):
        for num, games in enumerate(gameboard.findAll('td', attrs={'class': 'viBodyBorderNorm'})):
            for game_info in games.findAll('td'):  
                for date in game_info.findAll('span', attrs={'class': 'cellTextHot'}):
                    date = date.get_text().split()[0]
                    month = date.split('/')[0]
                    day = date.split('/')[1]

                    if(int(day[0]) == 0):
                        day = day[1:]

                    date = str(month) + '/' + str(day) + '/' + str(year)
                    dates_list.extend([date, date])

                for team_name in game_info.findAll('a', attrs={'class': 'tabletext'}):
                    teams_list.append(town_to_nickname_dict[team_name.get_text()])

            for num, spread in enumerate(games.findAll('td',attrs={'class': 
                                                                  ['viCellBg1 cellTextNorm cellBorderL1 center_text nowrap',
                                                                   'viCellBg1 cellTextHot cellBorderL1 center_text nowrap',
                                                                   'viCellBg2 cellTextNorm cellBorderL1 center_text nowrap',
                                                                   'viCellBg2 cellTextHot cellBorderL1 center_text nowrap']})):           
                spread_text = spread.get_text().strip()
                if spread_text == '' or spread_text == 'XXXX':
                    away_spread = np.nan
                    home_spread = np.nan
                elif(spread_text[4] == '+' or spread_text[4] == '-'):
                    away_spread = spread_text[0:4]
                    home_spread = spread_text[4:]
                else:
                    away_spread = spread_text[0:5]
                    home_spread = spread_text[5:]

                temp_away_list.append(away_spread)
                temp_home_list.append(home_spread)
                
                if len(temp_home_list) == 9:
                    book = book.append(pd.Series(temp_away_list, index=names), ignore_index=True)
                    book = book.append(pd.Series(temp_home_list, index=names), ignore_index=True)
                    temp_away_list = []
                    temp_home_list = []

    book['team'] = teams_list
    book['date'] = dates_list

    book = book[book.date <= df.date.max()]    # Only getting odds for games in current week
    odds = book[['team', 'odds']]
    
    df = pd.merge(odds, df)[['date', 'team', '538 win%', 'odds']]

    return df
def date_formatter(row):
    """Loads odds to spreadsheet
    Args:
        df: dataframe that odds will be written to
        year: year that will be used to gather the dates of the games
    Returns:
        dataframe of datawith odds
    """
    
    if row.date == '':
        return(row.date)
    else:
        split_date = row.date.split('/')
        day = split_date[1]
        day = day.zfill(2)
        date = split_date[0] + day + split_date[2]
        date = datetime.datetime.strptime(date, '%m%d%Y')
        date = date.strftime('%m/%d/%Y')
        return(date)
            
    
def combining_data(df_top, df_bottom):
    """Appends two dataframes
    Args:
        df_top: dataframe of data to go on top
        df_bottom: dataframe of data to go on bottom
    Returns:
        df: combined dataframe
    """
    cols = df_top.columns
    df = pd.concat([df_top, df_bottom], ignore_index=True, )
    df = df[cols]
    df.replace(np.nan, '', inplace=True)
    return(df)
def game_outcomes(df, week_num):
    """Finds winners and losers of games
    Args:
        df: dataframe of data
        week_num: week of games
    Returns:
        winners, losers: lists of winners and loser of games
    """
    year = 2019
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2019-nfl-predictions/games/?ex_cid=rrpromo'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    winners = []
    losers = []

    # Finding year of predictions
    for timestamp in soup.findAll('div', attrs={'class': 'container'}):
        for year in timestamp.findAll('div', attrs={'id': 'intro'}):
            year = int(year.h1.get_text()[0:5])

    # Gathering data
    for week in soup.findAll('section', attrs={'class': 'week'}):
        for week_text in week.findAll('h3', attrs={'class': 'h3'}):
            week_num_text = week_text.get_text()[5:]
            
        if week_num_text == str(week_num):
            for date in week.findAll('div', attrs={'class': 'days'}):
                for day in date.findAll('div', attrs={'class': 'day'}):  
                    for game in day.findAll('div', attrs={'class': 'game'}):

                        # Finding date of each game  
                        for h4 in day.findAll('h4', attrs={'class': 'h4'}):
                            date = h4.get_text()
                            date = date.split(', ')[1]
                            date = date[0:3] + date[4:] + ' ' + str(year)
                            date = datetime.datetime.strptime(date, '%b %d %Y')
                            date = date.strftime('%m/%d/%Y')

                        # Finding data for each game
                        for game_body in game.findAll('table', attrs={'class': 'game-body'}):           
                            for num_teams, matchup in enumerate(game_body.findAll('tr', attrs={'class': 'tr'})):
                                winner = matchup.find('td', attrs={'class': 'td text team winner'})
                                loser = matchup.find('td', attrs={'class': 'td text team loser'})

                                if winner:
                                    winners.append(winner.get_text().strip())
                                elif loser:
                                    losers.append(loser.get_text().strip())
                                else:
                                    continue
                                
    
    return winners, losers
def odds_checker(row):
    """Checks if odds for game are present
    Args:
        row: row of data
    Returns:
        
    """
    if row['odds'] != '':
        return(row['odds'])
    else:
        return(row['odds new'])  
def implied_probability(row):
    """Uses odds to determine implied probability
    Args:
        row: row of data from dataframe
    Returns:
        implied win probability if it exists
    """
    if 'implied' in row.index:
        return row['implied']
    elif row['odds'] == '':
        return('')
    elif row['odds'][0] == '+':
        return(round(100.0/(100+int(row['odds'][1:])), 2))
    elif row['odds'][0] == '-':
        return(round(int(row['odds'][1:])/(100.0+int(row['odds'][1:])), 2))
    else:
        return('')    
    
    
def pick(row, predictor):
    """Uses win% and odds to determine what team to pick
    Args:
        row: row of data from dataframe
        predictor: name of predicting column
    Returns:
        pick if there is one
    """
    if predictor + ' pick' in row.index:
        if row[predictor + ' pick'] == '':
            return ''
        else:    
            return(nickname_to_acronym_dict[row[predictor + ' pick']])
    elif row['implied'] == '':
        return('')
    elif (float(row[predictor + ' win%'][:-1])/100.0 > row['implied']):
        return(nickname_to_acronym_dict[row.team])
    else:
        return('')
    
def spreadsheet_formatter(df):
    """Formats spreadsheet
    Args:
        df: dataframe of data

    """
    if 'Unnamed: 0' in df.columns:
        df.drop('Unnamed: 0', axis=1, inplace=True)
        
    df.replace(np.nan, '', inplace=True)
    
    
def odds_formatter(row):
    """Formats odds
    Args:
        row: row of data from dataframe
    Returns:
        formatted odds
    """
    if row['odds'] == '':
        return(row['odds'])
    elif int(row['odds']) >= 100:
        return('+' + str(row['odds']))
    if int(row['odds']) <= -100:
        return(str(row['odds']))
    else:
        return(str(row['odds'])) 
    
def team_won_lost(row, winners, losers):
    """Determines team outcomes
    Args:
        row: row of data from dataframe
    Returns:
        updated entry for winner/loser
    """
    if row['team'] in winners:
        return('w')
    elif row['team'] in losers:
        return('l')
    else:
        return('')
     
def money_won_lost(row, predictor):
    """Determines amount won lost
    Args:
        row: row of data from dataframe
        predictor: name of predictor
    Returns:
        amount won/lost
    """
    if row[predictor + ' pick'] == '':
        return(0)
    else:
        if row['result'] == 'w':
            if row['odds'][0] == '+':
                return(int(row['odds'][1:]))
            else:
                return(100)
        elif row['result'] == 'l':
            if row['odds'][0] == '+':
                return(-100)
            else:
                return(int(row['odds']))
        else:
            return(0)
def week_finder(date):
    """Finds what week games are in
    Args:
        date: date in mm/dd/yyyy form
    Returns:
        week: week of games
    """
    if date < datetime.date(2019, 9, 10):
        week = 1
    elif date < datetime.date(2019, 9, 17):
        week = 2
    elif date < datetime.date(2019, 9, 24):
        week = 3
    elif date < datetime.date(2019, 10, 1):
        week = 4
    elif date < datetime.date(2019, 10, 8):
        week = 5
    elif date < datetime.date(2019, 10, 15):
        week = 6
    elif date < datetime.date(2019, 10, 22):
        week = 7
    elif date < datetime.date(2019, 10, 29):
        week = 8
    elif date < datetime.date(2019, 11, 5):
        week = 9
    elif date < datetime.date(2019, 11, 12):
        week = 10
    elif date < datetime.date(2019, 11, 19):
        week = 11
    elif date < datetime.date(2019, 11, 26):
        week = 12
    elif date < datetime.date(2019, 12, 3):
        week = 13
    elif date < datetime.date(2019, 12, 10):
        week = 14
    elif date < datetime.date(2019, 12, 17):
        week = 15
    elif date < datetime.date(2019, 12, 24):
        week = 16
    elif date < datetime.date(2019, 12, 31):
        week = 17
    else:
        week = 0
        
    return(week)
def writing_spreadsheet(df, filename):
    """Writing to spreadsheet
    Args:
        df: data
        file_name: path of file
    """
    df.to_csv(filename, index=False)

In [73]:
file_name = 'z:\\python projects\\NBA Game Outcome Spreadsheet.csv'

#df, year = initial_538_predictions(datetime.date.today().strftime('%m/%d/%Y'))
new_games, year = predictions_538(datetime.date.today().strftime('%m/%d/%Y'))
new_games = loading_odds(new_games, year)

new_games['implied'] = new_games.apply(lambda row: implied_probability(row), axis=1)
new_games['538 pick'] = new_games.apply(lambda row: pick(row, '538'), axis=1)
new_games
#datetime.date.today().strftime('%m/%d/%Y')
#writing_spreadsheet(df, file_name)

         date      team 538 win%  odds
0  10/22/2019  Pelicans      30%  +240
1               Raptors      70%  -300
2  10/22/2019    Lakers      43%  -160
3              Clippers      57%  +140


,date,team,538 win%,odds,implied,538 pick
0,10/22/2019,Pelicans,30%,+240,0.29,NO
1,,Raptors,70%,-300,0.75,
2,10/22/2019,Lakers,43%,-160,0.62,
3,,Clippers,57%,+140,0.42,LAC


In [ ]:
file_name = 'z:\\python projects\\NFL Game Outcome Spreadsheet.csv'

df = getting_spreadsheet(file_name) # Retrieving Spreadsheet
spreadsheet_formatter(df)
old_games, current_games = separating_games(df)

if not current_games.empty:
    current_games['odds'] = current_games.apply(lambda row: odds_formatter(row), axis=1)
    winners, losers = game_outcomes(current_games, week_num=week_finder(datetime.date.today()))
    current_games['result'] = current_games.apply(lambda row: team_won_lost(row, winners, losers), axis=1)
    current_games['538 w/l'] = current_games.apply(lambda row: money_won_lost(row, '538'), axis=1)
    current_games['Fox w/l'] = current_games.apply(lambda row: money_won_lost(row, 'Fox'), axis=1)
    #current_games['538 total'] = current_games['538 w/l'].sum()
    #current_games['Fox total'] = current_games['Fox w/l'].sum()
    df = combining_data(old_games, current_games)
  
df['date'] = df.apply(lambda row: date_formatter(row), axis=1)

if datetime.date.today().strftime('%m/%d/%Y') > df.date.max():
    new_games, year = predictions_538(week_num=week_finder(datetime.date.today()))
    new_games = predictions_fox(new_games)
    new_games = loading_odds(new_games, year)
    new_games['implied'] = new_games.apply(lambda row: implied_probability(row), axis=1)
    new_games['538 pick'] = new_games.apply(lambda row: pick(row, '538'), axis=1)
    new_games['Fox pick'] = new_games.apply(lambda row: pick(row, 'Fox'), axis=1)
    df = combining_data(df, new_games)
    
df['538 total'] = df['538 w/l'].sum()
df['Fox total'] = df['Fox w/l'].sum()
df
#writing_spreadsheet(df, file_name)